In [ ]:
!pip install squidpy scanpy matplotlib seaborn pandas numpy

In [ ]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

import scanpy as sc
import squidpy as sq

import os

In [ ]:
# Set the working directory
new_working_directory = r"C:\Users\leona\Documents\Documents Async\Master\JupyterProjects"
os.chdir(new_working_directory)


In [ ]:
#set data directory of the xenium output
data_directory = r"C:\Users\leona\Documents\Documents Async\Master\Xenium Files\output_BC53-66"

# Read the data directly from the specified directory
adata = sc.read_10x_h5(
    filename=os.path.join(data_directory, "cell_feature_matrix.h5")
)
df = pd.read_csv(
    os.path.join(data_directory, "cells.csv.gz")
)



In [ ]:
# Decompress the .csv.gz file if needed
df.to_csv(os.path.join(data_directory, "cells.csv"), index=False)
df = pd.read_csv(os.path.join(data_directory, "cells.csv"))

In [ ]:
# Set the index and update adata.obs
df.set_index(adata.obs_names, inplace=True)
adata.obs = df.copy()

In [ ]:
# Update adata.obsm with spatial coordinates
adata.obsm["spatial"] = adata.obs[["x_centroid", "y_centroid"]].copy().to_numpy()


In [ ]:
# Display adata.obs to verify the data
adata.obs

In [ ]:
adata

In [ ]:
#Quality control?
sc.pl.violin(adata, ['transcript_counts', 'total_counts', 'cell_area'], jitter=0.4, multi_panel=True)

In [ ]:
#exclude outliers?

In [ ]:
#Calculate the quality control metrics on the anndata.AnnData using scanpy.pp.calculate_qc_metrics.

sc.pp.calculate_qc_metrics(adata, percent_top=(10, 20, 50, 150), inplace=True)

In [ ]:
#The percentage of control probes and control codewords can be calculated from adata.obs
cprobes = (
    adata.obs["control_probe_counts"].sum() / adata.obs["total_counts"].sum() * 100
)
cwords = (
    adata.obs["control_codeword_counts"].sum() / adata.obs["total_counts"].sum() * 100
)
print(f"Negative DNA probe count % : {cprobes}")
print(f"Negative decoding count % : {cwords}")

In [ ]:
#Next we plot the distribution of total transcripts per cell, unique transcripts per cell, area of segmented cells and the ratio of nuclei area to their cells

fig, axs = plt.subplots(1, 4, figsize=(15, 4))

axs[0].set_title("Total transcripts per cell")
sns.histplot(
    adata.obs["total_counts"],
    kde=False,
    ax=axs[0],
)

axs[1].set_title("Unique transcripts per cell")
sns.histplot(
    adata.obs["n_genes_by_counts"],
    kde=False,
    ax=axs[1],
)


axs[2].set_title("Area of segmented cells")
sns.histplot(
    adata.obs["cell_area"],
    kde=False,
    ax=axs[2],
)

axs[3].set_title("Nucleus ratio")
sns.histplot(
    adata.obs["nucleus_area"] / adata.obs["cell_area"],
    kde=False,
    ax=axs[3],
)


In [ ]:
#Filter the cells based on the minimum number of counts required using scanpy.pp.filter_cells. Filter the genes based on the minimum number of cells required with scanpy.pp.filter_genes. The parameters for the both were specified based on the plots above. They were set to filter out the cells and genes with minimum counts and minimum cells respectively.
#Other filter criteria might be cell area, DAPI signal or a minimum of unique transcripts.

sc.pp.filter_cells(adata, min_counts=10)
sc.pp.filter_genes(adata, min_cells=5)

In [ ]:
#Normalize counts per cell using scanpy.pp.normalize_total.
#Logarithmize, do principal component analysis, compute a neighborhood graph of the observations using scanpy.pp.log1p, scanpy.pp.pca and scanpy.pp.neighbors respectively.
#Use scanpy.tl.umap to embed the neighborhood graph of the data and cluster the cells into subgroups employing scanpy.tl.leiden.

adata.layers["counts"] = adata.X.copy()
sc.pp.normalize_total(adata, inplace=True)
sc.pp.log1p(adata)
#sc.pp.pca(adata)
sc.pp.neighbors(adata, use_rep = 'X')
sc.tl.umap(adata)
sc.tl.leiden(adata)

#add resultion to leiden to reduce number of clusters?

In [ ]:
#Subplot with scatter plot in UMAP (Uniform Manifold Approximation and Projection) basis. 
#The embedded points were colored, respectively, according to the total counts, number of genes by counts, 
#and leiden clusters in each of the subplots. This gives us some idea of what the data looks like.

sc.pl.umap(
    adata,
    color=[
        "total_counts",
        "n_genes_by_counts",
        "leiden",
    ],
    legend_loc='on data',
    wspace=0.4,
)

In [ ]:
#try to identify clusters with celltype annotation genes
#how to use multiple annotation genes?

sc.pl.umap(adata, color=['EGFP-1','EGFP-2','EGFP-3', 'Col6a1', 'Col14a1', 'Col12a1', 'Col5a1', 'Col5a2', 'Col5a2', 'Fap', 'Cd4', 'Cd8a'], use_raw=False)

In [ ]:
sc.pl.umap(adata, color=['Cd3e', 'Cd4', 'Cd8a','EGFP-2', 'Col6a1', 'Col12a1', 'Col5a2', 'Fap', 'Cd68', 'Pecam1', 'Cd80', 'Kras', 'CreERT2-1', 'CreERT2-2', 'CreERT2-4'], use_raw=False)

In [ ]:
##try to extract most expressed genes per cluster based on rankes
sc.tl.rank_genes_groups(adata, "leiden", method="wilcoxon")

In [ ]:
sc.pl.rank_genes_groups_dotplot(adata, n_genes=8)

In [ ]:
sc.pl.rank_genes_groups_heatmap(adata, groups="5", n_genes=50, groupby="leiden")

In [ ]:
#Visualize on spatial coordinates
sq.pl.spatial_scatter(
    adata,
    library_id="spatial",
    shape=None,
    color=[
        "leiden",
    ],
    wspace=0.8,
)

##add specific gene after "leiden"


In [ ]:
#First, we need to compute a connectivity matrix from spatial coordinates to calculate the centrality scores. 
#We can use squidpy.gr.spatial_neighbors for this purpose. We use the coord_type="generic" based on the data 
#and the neighbors are classified with Delaunay triangulation by specifying delaunay=True.

sq.gr.spatial_neighbors(adata, coord_type="generic", delaunay=True)

In [ ]:
#Centrality scores are calculated with squidpy.gr.centrality_scores, with the Leiden groups as clusters.
sq.gr.centrality_scores(adata, cluster_key="leiden")

In [ ]:
#The results were visualized by plotting the average centrality, closeness centrality, and degree centrality using squidpy.pl.centrality_scores.
sq.pl.centrality_scores(adata, cluster_key="leiden", figsize=(16, 5))

In [ ]:
#We can further visualize tissue organization in spatial coordinates with squidpy.pl.spatial_scatter, 
#with an overlay of the expressed genes which were colored in consonance with the Leiden clusters.

adata_subsample = sc.pp.subsample(adata, fraction=0.5, copy=True)


In [ ]:
sq.gr.co_occurrence(
    adata_subsample,
    cluster_key="leiden",
)
sq.pl.co_occurrence(
    adata_subsample,
    cluster_key="leiden",
    clusters="12",
    figsize=(10, 10),
)
sq.pl.spatial_scatter(
    adata_subsample,
    color="leiden",
    shape=None,
    size=2,
)

In [ ]:
#This dataset contains cell type annotations in anndata.Anndata.obs which are used for calculation of the neighborhood enrichment. 
#We calculate the neighborhood enrichment score with squidpy.gr.nhood_enrichment.

sq.gr.nhood_enrichment(adata, cluster_key="leiden")

In [ ]:
#And visualize the results with squidpy.pl.nhood_enrichment.

fig, ax = plt.subplots(1, 2, figsize=(13, 7))
sq.pl.nhood_enrichment(
    adata,
    cluster_key="leiden",
    figsize=(8, 8),
    title="Neighborhood enrichment adata",
    ax=ax[0],
)
sq.pl.spatial_scatter(adata_subsample, color="leiden", shape=None, size=2, ax=ax[1])

In [ ]:
##The Moran’s I global spatial auto-correlation statistics evaluates whether features (i.e. genes) 
##shows a pattern that is clustered, dispersed or random in the tissue are under consideration.

#We can compute the Moran’s I score with squidpy.gr.spatial_autocorr and mode = 'moran'. 
#We first need to compute a spatial graph with squidpy.gr.spatial_neighbors. We will also subset the number of genes to evaluate.

sq.gr.spatial_neighbors(adata_subsample, coord_type="generic", delaunay=True)
sq.gr.spatial_autocorr(
    adata_subsample,
    mode="moran",
    n_perms=100,
    n_jobs=1,
)
adata_subsample.uns["moranI"].head(10)

In [ ]:
#We can visualize some of those genes with squidpy.pl.spatial_scatter. 
#We could also pass mode = 'geary' to compute a closely related auto-correlation statistic, 
#Geary’s C. See squidpy.gr.spatial_autocorr for more information.

#sq.pl.spatial_scatter(
    adata_subsample,
    library_id="spatial",
    color=[
        "KRT7",
        "FOXA1",
    ],
    shape=None,
    size=2,
    img=False,
)